#####################################################

#### To compile Go modules with C types to work with Python run:
#### go build -o main.so -buildmode=c-shared functions/main.go

#####################################################

In [ ]:
import os
import sys

import hail as hl
import pyspark


hail_jars = os.path.join(
    os.sep.join(sys.executable.split(os.sep)[:6]),
    "lib",
    f"python{sys.version_info.major}.{sys.version_info.minor}",
    "site-packages",
    "hail",
    "backend",
    "hail-all-spark.jar",
)

CONF = pyspark.SparkConf().setAll(
    [
        ("spark.master", f"local[{20}]"),
        ("spark.app.name", "Hail_demonstration"),
        ("spark.jars", hail_jars),
        ("spark.driver.extraClassPath", hail_jars),
        ("spark.driver.memory", "30g"),
        ("spark.executor.extraClassPath", "./hail-all-spark.jar"),
        ("spark.serializer", "org.apache.spark.serializer.KryoSerializer"),
        ("spark.kryo.registrator", "is.hail.kryo.HailKryoRegistrator"),
    ]
)

sc = pyspark.SparkContext(conf=CONF)

hl.init(default_reference="GRCh38", sc=sc)

Loading BokehJS ...

/home/phil/micromamba/envs/hail/lib/python3.12/site-packages/hail/context.py:354: UserWarning:

Using hl.init with a default_reference argument is deprecated. To set a default reference genome after initializing hail, call `hl.default_reference` with an argument to set the default reference genome.

pip-installed Hail requires additional configuration options in Spark referring
  to the path to the Hail Python module directory HAIL_DIR,
  e.g. /path/to/python/site-packages/hail:
    spark.jars=HAIL_DIR/backend/hail-all-spark.jar
    spark.driver.extraClassPath=HAIL_DIR/backend/hail-all-spark.jar
    spark.executor.extraClassPath=./hail-all-spark.jarRunning on Apache Spark version 3.5.6
SparkUI available at http://192.168.88.24:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.135-034ef3e08116
LOGGING: writing to /home/phil/GitHub/matrix_table_consumer/hail-20250822-2309-0.2.135-034ef3e08116.log


2025-08-22 23:09:56.256 Hail: WARN: You are trying to read ./data/test1.vcf with *ONE* core of parallelism. This will be very slow. If this file is block-gzipped (bgzip-ed), use force_bgz=True instead.
2025-08-22 23:09:58.360 Hail: INFO: scanning VCF for sortedness...
2025-08-22 23:10:00.400 Hail: INFO: Coerced sorted VCF - no additional import work to do
2025-08-22 23:10:02.577 Hail: WARN: You are trying to read ./data/test1.vcf with *ONE* core of parallelism. This will be very slow. If this file is block-gzipped (bgzip-ed), use force_bgz=True instead.
2025-08-22 23:10:07.322 Hail: WARN: You are trying to read ./data/test1.vcf with *ONE* core of parallelism. This will be very slow. If this file is block-gzipped (bgzip-ed), use force_bgz=True instead.
2025-08-22 23:10:37.552 Hail: INFO: wrote matrix table with 13 rows and 1 column in 1 partition to ./data/matrix_table.mt
2025-08-22 23:10:41.975 Hail: INFO: wrote matrix table with 13 rows and 1 column in 1 partition to ./data/matrix_tab

In [1]:
from matrix_table_consumer.matrix_table_consumer import MatrixTableConsumer, NUM_CPU

Loading BokehJS ...

In [2]:
# Downloaded from https://ftp.1000genomes.ebi.ac.uk/vol1/ftp/release/20130502/

vcf_big_path = "./data/ALL.chr1.phase3_shapeit2_mvncall_integrated_v5b.20130502.genotypes.vcf.gz"  # GRCh37
vcf_mini_path = "./data/test1.vcf"  # GRCh38

matrix_table_path = "./data/matrix_table.mt"
json_path = "./data/matrix.json"

In [5]:
mt = hl.import_vcf(
    vcf_mini_path,
    reference_genome="GRCh38",
    array_elements_required=False,
    force=True,
)
mt.show()

,,
,,'tumor'
locus,alleles,GT
locus<GRCh38>,array<str>,call
chr1:13100,"[""C"",""T""]",0/1
chr1:25734793,"[""C"",""T""]",0/1
chr1:37323930,"[""C"",""T""]",0/1
chr2:1234,"[""C"",""T""]",0/1
chr2:1235,"[""C"",""T""]",0/1
chr3:1234,"[""C"",""T""]",0/1
chr4:1234,"[""C"",""T""]",0/1


In [8]:
consumer = MatrixTableConsumer(
    vcf_path=vcf_mini_path, is_gzip=False, reference_genome="GRCh38"
)

# Save MatrixTable

In [9]:
content = {}
content = consumer.prepare_metadata_for_saving(json_path=json_path, mt=mt)

Extracting fields:   0%|          | 0/2 [00:00<?, ?it/s]

Extracting fields: 100%|██████████| 2/2 [00:00<00:00, 34.10it/s]


# Download MatrixTable

In [ ]:
mt_new = consumer.prepare_metadata_for_loading(json_path=json_path)

mt_new.write(matrix_table_path, overwrite=True)
mt_new.count()

Prepare metadata for loading: 100%|██████████| 3/3 [00:00<00:00, 325.63it/s]


(13, 1)

# Collect rows

In [3]:
consumer = MatrixTableConsumer(
    vcf_path=vcf_big_path, is_gzip=True, reference_genome="GRCh37"
)

In [4]:
rows = consumer.collect(num_rows=200_000, num_cpu=1)
rows

[{'QUAL': 100,
  'POS': 10177,
  'CHROM': '1',
  'ID': '.',
  'REF': 'A',
  'ALT': 'AC',
  'FILTER': 'PASS',
  'INFO': 'AC=2130;AF=0.425319;AN=5008;NS=2504;DP=103152;EAS_AF=0.3363;AMR_AF=0.3602;AFR_AF=0.4909;EUR_AF=0.4056;SAS_AF=0.4949;AA=|||unknown(NO_COVERAGE);VT=INDEL'},
 {'QUAL': 100,
  'POS': 10235,
  'CHROM': '1',
  'ID': '.',
  'REF': 'T',
  'ALT': 'TA',
  'FILTER': 'PASS',
  'INFO': 'AC=6;AF=0.00119808;AN=5008;NS=2504;DP=78015;EAS_AF=0;AMR_AF=0.0014;AFR_AF=0;EUR_AF=0;SAS_AF=0.0051;AA=|||unknown(NO_COVERAGE);VT=INDEL'},
 {'QUAL': 100,
  'POS': 10352,
  'CHROM': '1',
  'ID': '.',
  'REF': 'T',
  'ALT': 'TA',
  'FILTER': 'PASS',
  'INFO': 'AC=2191;AF=0.4375;AN=5008;NS=2504;DP=88915;EAS_AF=0.4306;AMR_AF=0.4107;AFR_AF=0.4788;EUR_AF=0.4264;SAS_AF=0.4192;AA=|||unknown(NO_COVERAGE);VT=INDEL'},
 {'QUAL': 100,
  'POS': 10505,
  'CHROM': '1',
  'ID': '.',
  'REF': 'A',
  'ALT': 'T',
  'FILTER': 'PASS',
  'INFO': 'AC=1;AF=0.000199681;AN=5008;NS=2504;DP=9632;EAS_AF=0;AMR_AF=0;AFR_AF=0.0008;

In [5]:
rows_count = consumer.count()
rows_count

6468094

In [ ]:
rows = consumer.collect_all(num_cpu=NUM_CPU)
rows[:2]

[23-08-2025 17:03:06] - INFO - Collecting data
[23-08-2025 17:03:06] - INFO - 0 lines read
[23-08-2025 17:03:06] - INFO - 50000 lines read
[23-08-2025 17:03:07] - INFO - 100000 lines read
[23-08-2025 17:03:08] - INFO - 150000 lines read
[23-08-2025 17:03:09] - INFO - 200000 lines read
[23-08-2025 17:03:10] - INFO - 250000 lines read
[23-08-2025 17:03:11] - INFO - 300000 lines read
[23-08-2025 17:03:11] - INFO - 350000 lines read
[23-08-2025 17:03:12] - INFO - 400000 lines read
[23-08-2025 17:03:13] - INFO - 450000 lines read


In [ ]:
print(len(rows))

In [7]:
rows_hail = consumer.convert_rows_to_hail(rows=rows)
rows_hail[:5]

Converting rows to hail: 100%|██████████| 200000/200000 [00:01<00:00, 124833.67it/s]


[Struct(locus=Locus(contig=1, position=10177, reference_genome=GRCh37), alleles=['A', 'AC'], rsid='.', qual=100, filters='PASS', info=Struct(info='AC=2130;AF=0.425319;AN=5008;NS=2504;DP=103152;EAS_AF=0.3363;AMR_AF=0.3602;AFR_AF=0.4909;EUR_AF=0.4056;SAS_AF=0.4949;AA=|||unknown(NO_COVERAGE);VT=INDEL'), entries=[]),
 Struct(locus=Locus(contig=1, position=10235, reference_genome=GRCh37), alleles=['T', 'TA'], rsid='.', qual=100, filters='PASS', info=Struct(info='AC=6;AF=0.00119808;AN=5008;NS=2504;DP=78015;EAS_AF=0;AMR_AF=0.0014;AFR_AF=0;EUR_AF=0;SAS_AF=0.0051;AA=|||unknown(NO_COVERAGE);VT=INDEL'), entries=[]),
 Struct(locus=Locus(contig=1, position=10352, reference_genome=GRCh37), alleles=['T', 'TA'], rsid='.', qual=100, filters='PASS', info=Struct(info='AC=2191;AF=0.4375;AN=5008;NS=2504;DP=88915;EAS_AF=0.4306;AMR_AF=0.4107;AFR_AF=0.4788;EUR_AF=0.4264;SAS_AF=0.4192;AA=|||unknown(NO_COVERAGE);VT=INDEL'), entries=[]),
 Struct(locus=Locus(contig=1, position=10505, reference_genome=GRCh37), all

In [8]:
t = consumer.create_hail_table(rows=rows_hail)
t.show()

,,,,,,
,,,,,info,
locus,alleles,rsid,qual,filters,info,entries
locus<GRCh37>,array<str>,str,int32,str,str,array<struct{}>
1:10177,"[""A"",""AC""]",""".""",100,"""PASS""","""AC=2130;AF=0.425319;AN=5008;NS=2504;DP=103152;EAS_AF=0.3363;AMR_AF=0.3602;AFR_AF=0.4909;EUR_AF=0.4056;SAS_AF=0.4949;AA=|||unknown(NO_COVERAGE);VT=INDEL""",[]
1:10235,"[""T"",""TA""]",""".""",100,"""PASS""","""AC=6;AF=0.00119808;AN=5008;NS=2504;DP=78015;EAS_AF=0;AMR_AF=0.0014;AFR_AF=0;EUR_AF=0;SAS_AF=0.0051;AA=|||unknown(NO_COVERAGE);VT=INDEL""",[]
1:10352,"[""T"",""TA""]",""".""",100,"""PASS""","""AC=2191;AF=0.4375;AN=5008;NS=2504;DP=88915;EAS_AF=0.4306;AMR_AF=0.4107;AFR_AF=0.4788;EUR_AF=0.4264;SAS_AF=0.4192;AA=|||unknown(NO_COVERAGE);VT=INDEL""",[]
1:10505,"[""A"",""T""]",""".""",100,"""PASS""","""AC=1;AF=0.000199681;AN=5008;NS=2504;DP=9632;EAS_AF=0;AMR_AF=0;AFR_AF=0.0008;EUR_AF=0;SAS_AF=0;AA=.|||;VT=SNP""",[]
1:10506,"[""C"",""G""]",""".""",100,"""PASS""","""AC=1;AF=0.000199681;AN=5008;NS=2504;DP=9676;EAS_AF=0;AMR_AF=0;AFR_AF=0.0008;EUR_AF=0;SAS_AF=0;AA=.|||;VT=SNP""",[]
1:10511,"[""G"",""A""]",""".""",100,"""PASS""","""AC=1;AF=0.000199681;AN=5008;NS=2504;DP=9869;EAS_AF=0;AMR_AF=0.0014;AFR_AF=0;EUR_AF=0;SAS_AF=0;AA=.|||;VT=SNP""",[]
1:10539,"[""C"",""A""]",""".""",100,"""PASS""","""AC=3;AF=0.000599042;AN=5008;NS=2504;DP=9203;EAS_AF=0;AMR_AF=0.0014;AFR_AF=0;EUR_AF=0.001;SAS_AF=0.001;AA=.|||;VT=SNP""",[]


# Create MatrixTable from Table

In [ ]:
mt_from_table = consumer.combine_hail_matrix_table_and_table(mt=mt, table=t)
mt_from_table.show()

In [ ]:
mt_from_table.count()